In [5]:
# TO DO: 
# markdown
# Add precipitation considerations

In [12]:
import pandas as pd
import numpy as np
import os
import gmaps
import requests
import sys
import json
from pprint import pprint

sys.path.insert(0,'../')
from config import g_key

gmaps.configure(g_key)

In [13]:
weather_df = pd.read_csv(os.path.join("..","weather_database","WeatherPy_Database.csv"), index_col=0)
weather_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Castro,CL,-42.4721,-73.7732,57.38,90,90,11.50,light intensity shower rain
1,Ilulissat,GL,69.2167,-51.1000,15.82,35,100,12.66,overcast clouds
2,Dongsheng,CN,39.8161,109.9776,16.03,53,5,14.03,clear sky
3,Lufilufi,WS,-13.8745,-171.5986,78.58,76,47,19.82,light rain
4,Longyearbyen,SJ,78.2186,15.6401,12.04,57,0,12.66,clear sky
...,...,...,...,...,...,...,...,...,...
661,Pedasi,PA,7.5333,-80.0333,84.07,77,98,13.27,overcast clouds
662,Orlik,RU,52.5178,99.8250,-15.59,92,100,2.59,overcast clouds
663,Sinnar,SD,13.5691,33.5672,71.83,18,99,10.31,overcast clouds
664,Parrita,CR,9.5201,-84.3225,91.17,71,49,4.05,scattered clouds


In [14]:
### disallow unreasonable min/max values
min_temp_threshold = 100
max_temp_threshold = 0

#Input strings for repeatability.
min_temp_input_str = f"Please specify a minimum temperature no higher than {min_temp_threshold}: "
not_numeric_str = f"Chosen value is not a number."
threshold_str = f"Chosen value is outside specified threshold."
min_higher_than_max_str = f"Your max temperature is higher than your min temperature."

min_temp = input(min_temp_input_str)

# iterate on input until a valid value is given
while min_temp.isnumeric() == False or float(min_temp) > min_temp_threshold:
    if min_temp.isnumeric() == False:
        print(not_numeric_str)
    elif float(min_temp) > min_temp_threshold:
        print(threshold_str)
    min_temp = input(min_temp_input_str)
    
# Replace max threshold with the min value (if higher)
max_temp_threshold = max(float(min_temp),float(max_temp_threshold))
max_temp_input_str = f"Please specify a maximum temperature no lower than {int(max_temp_threshold)}: "

max_temp = input(max_temp_input_str)

# iterate on input until a valid value is given
while max_temp.isnumeric() == False or float(max_temp) < max_temp_threshold:
    if max_temp.isnumeric() == False:
        print(not_numeric_str)
    elif float(max_temp) < max_temp_threshold:
        print(threshold_str)
    max_temp = input(max_temp_input_str)
    
# Generate data frame from preferred temperatures and remove blanks
vacation_cities_df = weather_df.loc[(weather_df["Max Temp"] >= float(min_temp)) & (weather_df["Max Temp"] <= float(max_temp))].dropna()

print("------------------------")
print("Cities identified based on search criteria:")
vacation_cities_df

Please specify a minimum temperature no higher than 100:  40
Please specify a maximum temperature no lower than 40:  70


------------------------
Cities identified based on search criteria:


,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Castro,CL,-42.4721,-73.7732,57.38,90,90,11.50,light intensity shower rain
5,Bredasdorp,ZA,-34.5322,20.0403,61.29,82,96,8.05,overcast clouds
9,Lagoa,PT,39.0500,-27.9833,58.69,70,73,19.64,broken clouds
12,Garowe,SO,8.4054,48.4845,69.53,83,8,12.77,clear sky
14,Punta Arenas,CL,-53.1500,-70.9167,51.91,46,40,21.85,scattered clouds
...,...,...,...,...,...,...,...,...,...
651,Malanje,AO,-9.5402,16.3410,63.12,97,9,1.83,light rain
652,Ukiah,US,39.1502,-123.2078,52.63,59,75,10.36,broken clouds
654,Chimoio,MZ,-19.1164,33.4833,68.92,99,100,3.27,moderate rain
658,Bandipur,IN,11.6663,76.6286,62.74,96,92,4.07,overcast clouds


In [20]:
gmaps_base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

vacation_cities_df["Hotel Name"] = ""

json_file_name = "vacation_search_api.json"

# writing json calls to separate file in order to facilitate exploratory analysis
with open(json_file_name,mode="w+") as json_file:

    json_file.write("[")
    
    for index, row in vacation_cities_df.iterrows():
        if index != 0:
            json_file.write(",\n")
                
        location = f'{vacation_cities_df.loc[index]["Lat"]},{vacation_cities_df.loc[index]["Long"]}'

        params = {'key':g_key,
                  'location':location,
                  'radius':5000,
                  'type':'logding'
                }

        try:
            gmaps_request = requests.get(url=gmaps_base_url, params=params)
            gmaps_json = gmaps_request.json()

            #results[0] just yields city name.
            hotel_name = gmaps_json['results'][1]['name']

            print(f"Identified {hotel_name} in {row['City']}, {row['Country']}")        
            vacation_cities_df.loc[index, "Hotel Name"] = hotel_name
                  
            # https://www.geeksforgeeks.org/reading-and-writing-json-to-a-file-in-python/
            json.dump(gmaps_json,json_file)
        except KeyError:
            print(f"No hotels found for {row['City']}, {row['Country']}")
        except IndexError:
            print(f"No hotels found for {row['City']}, {row['Country']}") 

    json_file.write("\n]")
        
vacation_cities_df = vacation_cities_df.dropna()
vacation_cities_df

Identified Hotel Esmeralda in Castro, CL
Identified Bredasdorp Country Manor in Bredasdorp, ZA
Identified Casa Das Faias in Lagoa, PT
Identified Scansom Bookshop & Library in Garowe, SO
Identified Hotel Hain in Punta Arenas, CL
Identified Super 8 by Wyndham Fortuna in Fortuna, US
Identified فندق ميس إيجيبت Miss Egypt Hotel in Asyut, EG
Identified Tu Casa in East London, ZA
Identified Observatory Guest House in Busselton, AU
Identified مدرسة راهنمایی شهید مدرس in Mehran, IR
Identified BON Hotel Waterfront Richards Bay in Richards Bay, ZA
Identified Bluff Homestead - Guesthouse & Campervan Park in Bluff, NZ
Identified MOGOK HILL HOTEL in Mogok, MM
Identified Xiajiyi in Dali, CN
Identified D & D Fence in Ardmore, US
Identified Aqua Hotel Thermal in Mosonmagyarovar, HU
Identified Holiday Inn Express & Suites Rolla - Univ of Missouri S&T, an IHG Hotel in Rolla, US
Identified Dog Rock Motel in Albany, AU
Identified Hotel Am Kurpark in Schonebeck, DE
Identified Albatross Hotel in Ushuaia, AR


,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Castro,CL,-42.4721,-73.7732,57.38,90,90,11.50,light intensity shower rain,Hotel Esmeralda
5,Bredasdorp,ZA,-34.5322,20.0403,61.29,82,96,8.05,overcast clouds,Bredasdorp Country Manor
9,Lagoa,PT,39.0500,-27.9833,58.69,70,73,19.64,broken clouds,Casa Das Faias
12,Garowe,SO,8.4054,48.4845,69.53,83,8,12.77,clear sky,Scansom Bookshop & Library
14,Punta Arenas,CL,-53.1500,-70.9167,51.91,46,40,21.85,scattered clouds,Hotel Hain
...,...,...,...,...,...,...,...,...,...,...
651,Malanje,AO,-9.5402,16.3410,63.12,97,9,1.83,light rain,"Njinga Center - Convention, Culture and Leisure"
652,Ukiah,US,39.1502,-123.2078,52.63,59,75,10.36,broken clouds,Hampton Inn Ukiah
654,Chimoio,MZ,-19.1164,33.4833,68.92,99,100,3.27,moderate rain,Faculty of Engineering
658,Bandipur,IN,11.6663,76.6286,62.74,96,92,4.07,overcast clouds,OYO 38131 Bandipur Khans Resort


In [10]:
# reset my index before you forget.  Also don't hardcode this csv.
vacation_cities_df.to_csv("WeatherPy_vacation.csv")

In [ ]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]

coordinates_df = vacation_cities_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure(center=(0.0, 0.0), 
                   zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig